<img src="images/dask_horizontal_black.gif" alt="Dask Icon" width="450"/>
<img src="images/pandas_logo.png" alt="Pandsas Icon" width="450"/>

# Gotcha's from Pandas to Dask  
<div style="text-align:right"><span style="color:Blue; font-family:Georgia; font-size:1em;">Sephi Berry: js.berry@gmail.com</span></div>

https://github.com/sephib/dask_pyconil2019

This notebook highlights some key differences when transfering code from `Pandas` to run in a `Dask` environment.  
Most issues have a link to the [Dask documentation](https://docs.dask.org/en/latest/) for additional information.

# Agenda  
1. Intro to `Dask` framework
2. Basic setup  `Dask Client`
3. Dask.dataframe
4. Data manipulation
5. Read/Write files
6. Advanced `groupby`
7. Debugging


[![dask graph outline](images/dask_graph_outline.jpg)](daskgraph)

<img src="images\dask-dataframes.png" alt="Dask dataframes" width="450"/>


Dask is composed of two parts:

1. *Dynamic task scheduling* optimized for computation. This is similar to Airflow, Luigi, Celery, or Make, but optimized for interactive computational workloads.
2. *“Big Data” collections* like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of dynamic task schedulers.

[link to documentation](https://docs.dask.org/en/latest/)

Dask emphasizes the following virtues:

* Familiar: Provides parallelized NumPy array and Pandas DataFrame objects
* Flexible: Provides a task scheduling interface for more custom workloads and integration with other projects.
* Native: Enables distributed computing in pure Python with access to the PyData stack.
* Fast: Operates with low overhead, low latency, and minimal serialization necessary for fast numerical algorithms
* Scales up: Runs resiliently on clusters with 1000s of cores
* Scales down: Trivial to set up and run on a laptop in a single process
* Responsive: Designed with interactive computing in mind, it provides rapid feedback and diagnostics to aid humans


See the [dask.distributed documentation (separate website)](https://distributed.dask.org/en/latest/) for more technical information on Dask’s distributed scheduler.

# [Why Dask?](https://docs.dask.org/en/latest/why.html)
* Big Data (data doesn't fit into RAM)
* Scales from single comptuer out to clusters
* Familiar with `Pandas` API
* Responsive feedback (live dashboard)
* ...

# Agenda  
1. Intro to `Dask` framework  
**Basic setup  `Dask Client`**
3. Dask.dataframe
4. Data manipulation
5. Read/Write files
6. Advanced `groupby`
7. Debugging

In [1]:
# since Dask is activly beeing developed - the current example is running with the below version
import dask
import dask.dataframe as dd
import pandas as pd
print(f'Dask versoin: {dask.__version__}')
print(f'Pandas versoin: {pd.__version__}')

Dask versoin: 1.2.0
Pandas versoin: 0.24.2


## Dask `Distributed` scheduler  

In [2]:
from dask.distributed import Client 
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client = Client()

## Expose Dask Client for Dashboard

In [3]:
client

Client Scheduler: tcp://127.0.0.1:62957 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.50 GB


![Dask Dashboard](images\DaskDashboard2.png)


![Dask Dashboard Graph](images\graphdask2.png)

## When running code within a script use a `context manager`  
```python   
if __name__ == '__main__':
    with Client() as client:
        # do something
```

* see question in [stack overflow](https://stackoverflow.com/a/53520917/5817977)  
* In order to get url dashboard use [inner function ](https://github.com/dask/distributed/issues/2083#issue-337057906)  


Starting the Dask Client is optional.  In this example we are running on a `LocalCluster`, this  will also provide a dashboard which is useful to gain insight on the computation.  
For additional information on [Dask Client see documentation](https://docs.dask.org/en/latest/setup.html?highlight=client#setup)  

The link to the dashboard will become visible when you create a client (as shown below).  
When running in `Jupyter Lab` an [extenstion](https://github.com/dask/dask-labextension) can be installed to be able to view the various dashboard widgets. 

See [documentation for addtional cluster configuration](http://distributed.dask.org/en/latest/local-cluster.html)

# Agenda  
1. Intro to `Dask` framework  
2. Basic setup  `Dask Client`  
**3. Dask.dataframe**
4. Data manipulation
5. Read/Write files
6. Advanced `groupby`
7. Debugging

#### Create 2 DataFrames for comparison: 
<img src="images/Sleeping-snake.jpg" alt="Dask lazy dataframes" width="100"/>

In [3]:
ddf = dask.datasets.timeseries() #  Dask comes with builtin dataset samples, we will use this sample for our example. 
print(type(ddf))
ddf

<class 'dask.dataframe.core.DataFrame'>


,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In order to see the result we need to run [compute()](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.compute) 
 (or `head()` which runs under the hood compute()) )

In [5]:
ddf.compute()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398
2000-01-01 00:00:02,919,Alice,0.924705,0.232565
2000-01-01 00:00:03,1031,Victor,-0.480341,-0.105972
2000-01-01 00:00:04,1007,Ray,0.800868,-0.389520
2000-01-01 00:00:05,947,Zelda,-0.816978,-0.876242
2000-01-01 00:00:06,1089,Edith,0.027430,0.033384
2000-01-01 00:00:07,1010,Laura,0.118945,-0.570797
2000-01-01 00:00:08,1006,Michael,-0.479729,-0.340029


## Pandas
In order to create a `Pandas` dataframe we can use the `compute()` 

In [6]:
pdf = ddf.compute()  
print(type(pdf))
pdf

<class 'pandas.core.frame.DataFrame'>


,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398
2000-01-01 00:00:02,919,Alice,0.924705,0.232565
2000-01-01 00:00:03,1031,Victor,-0.480341,-0.105972
2000-01-01 00:00:04,1007,Ray,0.800868,-0.389520
2000-01-01 00:00:05,947,Zelda,-0.816978,-0.876242
2000-01-01 00:00:06,1089,Edith,0.027430,0.033384
2000-01-01 00:00:07,1010,Laura,0.118945,-0.570797
2000-01-01 00:00:08,1006,Michael,-0.479729,-0.340029


## Creating a `Dask dataframe` from `Pandas`

In order to utilize `Dask` capablities on an existing `Pandas dataframe` (pdf) we need to convert the `Pandas dataframe` into a `Dask dataframe` (ddf)  with the [from_pandas](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_pandas) method. 
You must supply the number of `partitions` or `chunksize` that will be used to generate the dask dataframe

In [7]:
ddf2 = dd.from_pandas(pdf, npartitions=10)
print(type(ddf2))
ddf2 

<class 'dask.dataframe.core.DataFrame'>


,id,name,x,y
npartitions=10,,,,
2000-01-01 00:00:00,int32,object,float64,float64
2000-01-04 00:00:00,...,...,...,...
...,...,...,...,...
2000-01-28 00:00:00,...,...,...,...
2000-01-30 23:59:59,...,...,...,...


## Partitions in Dask Dataframes

Notice that when we created a `Dask dataframe` we needed to supply an argument of `npartitions`.  
The number of partitions will assist `Dask` on how it's going to parallelize the computation.  
Each partition is a *separate* dataframe. For additional information see [partition documentation](https://docs.dask.org/en/latest/dataframe-design.html?highlight=meta%20utils#partitions)  



Using `reset_index()` method we can examin the partitions:  

In [8]:
pdf2 = pdf.reset_index()
pdf2.loc[0] # Only 1 row

timestamp    2000-01-01 00:00:00
id                          1039
name                         Bob
x                      -0.465773
y                      -0.204463
Name: 0, dtype: object

Now lets look at a `Dask` dataframe

In [10]:
ddf2 = ddf2.reset_index() 
ddf2.loc[0].compute()  # each partition has an index=0

,index,timestamp,id,name,x,y
0,0,2000-01-01,1039,Bob,-0.465773,-0.204463
0,0,2000-01-04,1024,Norbert,0.141641,-0.952728
0,0,2000-01-07,987,Tim,0.892242,-0.776087
0,0,2000-01-10,1021,Jerry,0.389869,0.536214
0,0,2000-01-13,960,Ray,-0.042439,-0.035537
0,0,2000-01-16,1026,Ingrid,-0.516899,0.090226
0,0,2000-01-19,1004,Jerry,0.402262,0.427229
0,0,2000-01-22,983,Kevin,-0.463926,-0.399068
0,0,2000-01-25,991,Bob,-0.183379,-0.058577
0,0,2000-01-28,964,Wendy,-0.345190,0.950012


```python
ddf2=dd.from_pandas(pdf, npartitions=10)
```

## dataframe.shape  
`Dask` is lazy- we cannot get the full shape before running `len`

In [11]:
print(f'Pandas shape: {pdf.shape}')
print('---------------------------')
print(f'Dask lazy shape: {ddf.shape}') 

Pandas shape: (2592000, 4)
---------------------------
Dask lazy shape: (Delayed('int-0fab9096-a071-46c4-b148-8628ac493142'), 4)


In [12]:
print(f'Dask computed shape: {len(ddf.index):,}')  # expensive

Dask computed shape: 2,592,000


Now that we have a `dask` (ddf) and a `pandas` (pdf) dataframe we can start to compair the interactions with them.

# Agenda  
1. Intro to `Dask` framework  
2. Basic setup  `Dask Client`
3. Dask.dataframe  
**4. Data manipulation**
5. Read/Write files
6. Advanced `groupby`
7. Debugging

# Moving from Update to Insert/Delete

![inplaceTrue](images/inplace_true.png "inplace_true")


Dask does not update - thus there are no arguments such as `inplace=True` which exist in Pandas.  
For more detials see [issue#653 on github](https://github.com/dask/dask/issues/653)

### Rename Columns

In [13]:
# Pandas 
print(pdf.columns)
pdf.rename(columns={'id':'ID'}, inplace=True)
pdf.columns

Index(['id', 'name', 'x', 'y'], dtype='object')


Index(['ID', 'name', 'x', 'y'], dtype='object')

In [ ]:
# Dask - Error
# ddf.rename(columns={'id':'ID'}, inplace=True)
# ddf.columns

'''
---------------------------------------------------------------------------  
TypeError                                 Traceback (most recent call last)  
<ipython-input-12-3e70ff3a549e> in <module>  
      1 # Dask - Error  
----> 2 ddf.rename(columns={'id':'ID'}, inplace=True)  
      3 ddf.columns  
TypeError: rename() got an unexpected keyword argument 'inplace'  
'''

* using `inplace=True` is *not* considerd to be *best practice*. 

In [14]:
# Dask or Pandas
print(ddf.columns)
ddf = ddf.rename(columns={'id':'ID'})
ddf.columns

Index(['id', 'name', 'x', 'y'], dtype='object')


Index(['ID', 'name', 'x', 'y'], dtype='object')

### loc - Pandas

In [15]:
mask_cond = (pdf['x']>0.5) & (pdf['x']<0.8)
pdf.loc[mask_cond, ['y']] = pdf['y']* 100
pdf[mask_cond].head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:09,994,Edith,0.658423,-3.203189
2000-01-01 00:00:20,1001,Sarah,0.508488,-29.411868


In [ ]:
# Error
# cond_dask = (ddf['x']>0.5) & (ddf['x']<0.8)
# ddf.loc[cond_dask, ['y']] = ddf['y']* 100

'''
> TypeError                                 Traceback (most recent call last)  
> <ipython-input-16-2bbb2ae570bd> in <module> 
>       2 # Error  
> ----> 3 ddf.loc[cond_dask, ['y']] = ddf['y']* 100  
> TypeError: '_LocIndexer' object does not support item assignment  
'''

### Dask - use mask/where

In [17]:
# Pandas
mask_cond_p = (pdf['x']>0.5) & (pdf['x']<0.8)
pdf['y'] = pdf['y'].mask(cond=mask_cond_p, other=pdf['y']* 100)
#Dask
mask_cond_d = (ddf['x']>0.5) & (ddf['x']<0.8)
ddf['y'] = ddf['y'].mask(cond=mask_cond_d, other=ddf['y']* 100)
print(f'Pandas: {pdf[mask_cond].head(1)}')
print(f'----------------------------------------------------------------')
print(f'Dask: {ddf[mask_cond].head(1)}') 

Pandas:                       ID   name         x             y
timestamp                                              
2000-01-01 00:00:09  994  Edith  0.658423 -32031.886786
----------------------------------------------------------------
Dask:                       ID   name         x           y
timestamp                                            
2000-01-01 00:00:09  994  Edith  0.658423 -320.318868


[dask mask documentation](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.mask)

## Meta argument

> `meta` is the prescription of the names/types of the computation output   
[see stack overflow answer](https://stackoverflow.com/questions/44432868/dask-dataframe-apply-meta)

![crystal python](images/crystalBallsnake.png "crystal snake")
Since `Dask` creates a DAG for the computation it requires to understand what are the outputs of each calculation (see [meta documentation](https://docs.dask.org/en/latest/dataframe-design.html?highlight=meta%20utils#metadata))

In [18]:
pdf['initials'] = pdf['name'].apply(lambda x: x[0]+x[1])
pdf.head(2)

,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463,Bo
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398,Ze


In [19]:
ddf['initials'] = ddf['name'].apply(lambda x: x[0]+x[1])
ddf.head(2)

c:\users\jsber\.virtualenvs\dask_pyconil2019-dwj2t3bv\lib\site-packages\dask\dataframe\core.py:2345: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('name', 'object'))

  warnings.warn(meta_warning(meta))


,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463,Bo
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398,Ze


#### Introducing meta argument

In [20]:
# Describe the outcome type of the calculation
meta_cal = pd.Series(object, name='initials')
ddf['initials'] = ddf['name'].apply(lambda x: x[0]+x[1]
                                    , meta = meta_cal)
ddf.head(2)

,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463,Bo
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398,Ze


In [21]:
def func(row, col1, col2):
    if (row[col1]> 0):  return row[col1] * 1000  
    else:        return row[col2] * -1
ddf['z'] = ddf.apply(func, args=('x', 'y'), axis=1
                     , meta=('float'))
ddf.head(2)

,ID,name,x,y,initials,z
timestamp,,,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463,Bo,0.204463
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398,Ze,304.994839


### Map partitions
* We can supply an ad-hoc function to run on each partition using the [map_partitions](https://dask.readthedocs.io/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions) method.   
Mainly useful for functions that are not implemented in `Dask` or `Pandas` . 
* Finally we can return a new `dataframe` which needs to be described in the `meta` argument  
The function could also include arguments.

In [22]:
import numpy as np
def func2(df, coor_x, coor_y, drop_cols):
    df['dist'] =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  
                           +  (df[coor_y] - df[coor_y].shift())**2 )
    return df.drop(drop_cols, axis=1)

ddf2 = ddf.map_partitions(func2
                          , coor_x='x'
                          , coor_y='y'
                          , drop_cols=['initials', 'z']
                          , meta=pd.DataFrame({'ID':'i8'
                                              , 'name':str
                                              , 'x':'f8'
                                              , 'y':'f8'                                              
                                              , 'dist':'f8'}, index=[0]))
ddf2.head()

,ID,name,x,y,dist
timestamp,,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463,NaN
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398,0.836849
2000-01-01 00:00:02,919,Alice,0.924705,0.232565,0.982930
2000-01-01 00:00:03,1031,Victor,-0.480341,-0.105972,1.445254
2000-01-01 00:00:04,1007,Ray,0.800868,-0.389520,1.312210


### Convert index into DateTime column

In [23]:
  # Only Pandas
pdf = pdf.assign(times=pd.to_datetime(pdf.index).time)
pdf.head(2)

,ID,name,x,y,initials,times
timestamp,,,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463,Bo,00:00:00
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398,Ze,00:00:01


In [24]:
# Dask or Pandas
ddf = ddf.assign(times=ddf.index.astype('M8[ns]'))
ddf['times'] = ddf['times'].dt.time
ddf =client.persist(ddf)
ddf.head(2)

,ID,name,x,y,initials,z,times
timestamp,,,,,,,
2000-01-01 00:00:00,1039,Bob,-0.465773,-0.204463,Bo,0.204463,00:00:00
2000-01-01 00:00:01,983,Zelda,0.304995,-0.530398,Ze,304.994839,00:00:01


## Drop NA on column

In [25]:
pdf = pdf.drop(labels=['initials'],axis=1)
ddf = ddf.drop(labels=['initials','z'],axis=1) 

In [26]:
pdf = pdf.assign(colna = None)
print(f'pandas: {pdf.head(1)}')
ddf = ddf.assign(colna = None)
print(f'dask: {ddf.head(1)}')

pandas:               ID name         x         y     times colna
timestamp                                                
2000-01-01  1039  Bob -0.465773 -0.204463  00:00:00  None
dask:               ID name         x         y     times colna
timestamp                                                
2000-01-01  1039  Bob -0.465773 -0.204463  00:00:00  None


In odrer for `Dask` to drop a column with all `na` we need to assist the graph

In [27]:
pdf = pdf.dropna(axis=1, how='all')
pdf.head(1)

,ID,name,x,y,times
timestamp,,,,,
2000-01-01,1039,Bob,-0.465773,-0.204463,00:00:00


In [28]:
# check if all values in column are Null - expensive
if ddf.colna.isnull().all() == True:   
    ddf = ddf.drop(labels=['colna'],axis=1)
ddf.head(1)

,ID,name,x,y,times
timestamp,,,,,
2000-01-01,1039,Bob,-0.465773,-0.204463,00:00:00


##  Reset Index

In [ ]:
# Pandas
pdf = pdf.reset_index(drop=True)
pdf.head(1)

In [ ]:
# Dask
ddf = ddf.reset_index()
ddf = ddf.drop(labels=['timestamp'], axis=1 )
ddf.head(1)

# Agenda  
1. Intro to `Dask` framework  
2. Basic setup  `Dask Client`
3. Dask.dataframe
4. Data manipulation  
**5. Read/Write files**
6. Advanced `groupby`
7. Debugging

# Read / Save files

* When working with `pandas` and `dask` preferable try and work with [parquet](https://docs.dask.org/en/latest/dataframe-best-practices.html?highlight=parquet#store-data-in-apache-parquet-format).  
* Even so when working with `Dask` - the files can be read with multiple workers .  
* Most `kwargs` are applicable for reading and writing files   
e.g. 
ddf = dd.read_csv('data/pd2dd/ddf*.csv', compression='gzip', header=False).  

However some are not available such as  `nrows`.

[see documentaion](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.to_csv) (including the option for output file naming).

## Save files

In [36]:
%%time
# Pandas
from pathlib import Path
output_dir_file = Path('data/pdf_single_file.csv')
output_dir_file.parent.mkdir(parents=True, exist_ok=True)
pdf.to_csv(output_dir_file)

Wall time: 25.3 s


In [37]:
list(output_dir_file.parent.glob('*.csv'))

[WindowsPath('data/pdf_single_file.csv')]

`Dask`
Notice the '*' to allow for multiple file renaming. 



In [38]:
%%time
# Dask
output_dask_dir = Path('data/pd2dd/')
output_dask_dir.mkdir(parents=True, exist_ok=True)
ddf.to_csv(f'{output_dask_dir}/ddf*.csv', index = False)

Wall time: 19.1 s


To find the number of partitions which will determine the number of output files use [dask.dataframe.npartitions](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.npartitions)  

In [39]:
ddf.npartitions

30

In [40]:
list(Path(output_dask_dir).glob('*.csv'))

[WindowsPath('data/pd2dd/ddf00.csv'),
 WindowsPath('data/pd2dd/ddf01.csv'),
 WindowsPath('data/pd2dd/ddf02.csv'),
 WindowsPath('data/pd2dd/ddf03.csv'),
 WindowsPath('data/pd2dd/ddf04.csv'),
 WindowsPath('data/pd2dd/ddf05.csv'),
 WindowsPath('data/pd2dd/ddf06.csv'),
 WindowsPath('data/pd2dd/ddf07.csv'),
 WindowsPath('data/pd2dd/ddf08.csv'),
 WindowsPath('data/pd2dd/ddf09.csv'),
 WindowsPath('data/pd2dd/ddf10.csv'),
 WindowsPath('data/pd2dd/ddf11.csv'),
 WindowsPath('data/pd2dd/ddf12.csv'),
 WindowsPath('data/pd2dd/ddf13.csv'),
 WindowsPath('data/pd2dd/ddf14.csv'),
 WindowsPath('data/pd2dd/ddf15.csv'),
 WindowsPath('data/pd2dd/ddf16.csv'),
 WindowsPath('data/pd2dd/ddf17.csv'),
 WindowsPath('data/pd2dd/ddf18.csv'),
 WindowsPath('data/pd2dd/ddf19.csv'),
 WindowsPath('data/pd2dd/ddf20.csv'),
 WindowsPath('data/pd2dd/ddf21.csv'),
 WindowsPath('data/pd2dd/ddf22.csv'),
 WindowsPath('data/pd2dd/ddf23.csv'),
 WindowsPath('data/pd2dd/ddf24.csv'),
 WindowsPath('data/pd2dd/ddf25.csv'),
 WindowsPath

To change the number of output files use [repartition](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.repartition) which is an expensive operation.

## Read files - Pandas

For `pandas` it is possible to iterate and concat the files [see answer from stack overflow](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe).

In [41]:
%%time
# Pandas
dir_path = Path(r'data/pd2dd')
concat_df = pd.concat([pd.read_csv(f) 
                       for f in list(dir_path.glob('*.csv'))])
len(concat_df)

Wall time: 6.15 s


## Read files - Dask

In [42]:
%%time
_ddf = dd.read_csv('data/pd2dd/ddf*.csv')
_ddf

Wall time: 62.5 ms


In [43]:
%%time
_ddf = dd.read_csv('data/pd2dd/ddf*.csv')
len(_ddf)

Wall time: 3.48 s


 ## Consider using client.persist()

![dask graph](images\graphdask2.png "dask graph")

### client.persist()

In [44]:
_ddf = dd.read_csv('data/pd2dd/ddf*.csv')
# do some filter
_ddf = client.persist(_ddf)
# do some computations
_ddf.head(2)

,ID,name,x,y,times
0,1039,Bob,-0.465773,-0.204463,00:00:00
1,983,Zelda,0.304995,-0.530398,00:00:01


In [45]:
_ddf.head(2)

,ID,name,x,y,times
0,1039,Bob,-0.465773,-0.204463,00:00:00
1,983,Zelda,0.304995,-0.530398,00:00:01


Since Dask is lazy - it may run the **entire** graph/DAG (again) even if it already run part of the calculation in a previous cell.  Thus use [persist](https://docs.dask.org/en/latest/dataframe-best-practices.html?highlight=parquet#persist-intelligently) to keep the results in memory  
Additional information can be read in this [stackoverflow issue](https://stackoverflow.com/questions/45941528/how-to-efficiently-send-a-large-numpy-array-to-the-cluster-with-dask-array/45941529#45941529) or see an exampel in [this post](http://matthewrocklin.com/blog/work/2017/01/12/dask-dataframes)   
This concept should also  be used when running a code within a script (rather then a jupyter notebook) which incoperates loops within the code.

# Agenda  
1. Intro to `Dask` framework  
2. Basic setup  `Dask Client`
3. Dask.dataframe
4. Data manipulation
5. Read/Write files  
**6. Advanced `groupby`**
7. Debugging

# Group By - custom aggregations
Try to eliminate the use of `groupby.apply`  
Example: `groupby` columns into unique list.

In [46]:
# prepare pandas dataframe
pdf = pdf.assign(time=pd.to_datetime(pdf.index).time)
pdf['seconds'] = pdf.time.astype(str).str[-2:]
cols_for_demo =['name', 'ID','seconds']
pdf[cols_for_demo].head()

,name,ID,seconds
timestamp,,,
2000-01-01 00:00:00,Bob,1039,00
2000-01-01 00:00:01,Zelda,983,01
2000-01-01 00:00:02,Alice,919,02
2000-01-01 00:00:03,Victor,1031,03
2000-01-01 00:00:04,Ray,1007,04


In addition to the [groupby notebook example](https://github.com/dask/dask-examples/blob/master/dataframes/02-groupby.ipynb)  - 

### Pandas

In [47]:
%%time
pdf_gb = pdf.groupby(pdf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [pdf_gb[att_col_gr].apply
               (lambda x: list(set(x.to_list()))) 
               for att_col_gr in gp_col]
df_edge_att = pdf_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')      

Wall time: 1.8 s


In [48]:
df_edge_att.head(2)

,Weight,ID,seconds
name,,,
Alice,99484,"[1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...","[46, 16, 25, 20, 02, 13, 18, 30, 19, 55, 34, 4..."
Bob,99280,"[1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...","[46, 16, 25, 20, 02, 13, 18, 34, 55, 30, 19, 4..."


In any case sometimes using Pandas is more efficiante (assuming that you can load all the data into the RAM).  
In this case Pandas is faster

In [49]:
def set_list_att(x: dd.Series):
        return list(set([item for item in x.values]))
ddf['seconds'] = ddf.times.astype(str).str[-2:]
ddf = client.persist(ddf)
ddf[cols_for_demo].head(2)

,name,ID,seconds
timestamp,,,
2000-01-01 00:00:00,Bob,1039,00
2000-01-01 00:00:01,Zelda,983,01


In [50]:
%%time
df_gb = ddf.groupby(ddf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [df_gb[att_col_gr].apply(set_list_att
                ,meta=pd.Series(dtype='object', name=f'{att_col_gr}_att')) 
               for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
    df_edge_att = df_edge_att.join(ser.to_frame(), how='left')
df_edge_att.head(2)

Wall time: 26.6 s


Using [dask custom aggregation](https://docs.dask.org/en/latest/dataframe-api.html?highlight=dropna#dask.dataframe.groupby.Aggregation) is considerably better

In [51]:
import itertools
custom_agg = dd.Aggregation(
    'custom_agg', 
    lambda s: s.apply(set), 
    lambda s: s.apply(lambda chunks: list(set(itertools.chain.from_iterable(chunks)))),)

In [52]:
%%time
df_gb = ddf.groupby(ddf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [df_gb[att_col_gr].agg(custom_agg) for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')
df_edge_att.head(2)  

Wall time: 3.71 s


In [ ]:
df_edge_att.head()

# Agenda  
1. Intro to `Dask` framework  
2. Basic setup  `Dask Client`
3. Dask.dataframe
4. Data manipulation
5. Read/Write files
6. Advanced `groupby`  
**7. Debugging**

## [Debugging](https://docs.dask.org/en/latest/debugging.html)
Debugging may be challenging...
1. Run code without client 
2. Use Dashboard profiler
3. Verify integrity of DAG

## Corrupted DAG

In [53]:
# reset dataframe
ddf = dask.datasets.timeseries()
ddf.head(1)

,id,name,x,y
timestamp,,,,
2000-01-01,953,Ray,0.671122,0.438853


In [54]:
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())^2  
                     +  (df[coor_y] - df[coor_y].shift())^2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))

In [55]:
# returns an error because of ^2
ddf.head()

TypeError: unsupported operand type(s) for ^: 'float' and 'bool'

* Even if the function is corrected the DAG is corrupted

In [56]:
# Still results with an error
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  
                     +  (df[coor_y] - df[coor_y].shift())**2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))
ddf.head(2)

TypeError: unsupported operand type(s) for ^: 'float' and 'bool'

Need to reset the dataframe

In [57]:
ddf = dask.datasets.timeseries()
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  
                     +  (df[coor_y] - df[coor_y].shift())**2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))
ddf.head(2)

,id,name,x,y,col
timestamp,,,,,
2000-01-01 00:00:00,988,Oliver,0.294565,0.2014,NaN
2000-01-01 00:00:01,974,Ray,-0.073221,-0.4376,0.737284


### Summary
1. Similar to `Pandas` API (vs. `pyspark`)
2. `Dask` is lazy / efficient (parallel computing) 
3. Beware of:
  * missing functionalities from `Pandas` API
  * corrupted DAGs
4. Flexible `client` environments 
5. Bonus - dashboard and other goodies from `Dask` 

<div style="text-align:right"><span style="color:Blue; font-family:Georgia; font-size:1em;">js.berry@gmail.com</span></div>  
<div style="text-align:left"><span style="color:Black; font-family:Georgia; font-size:1em;">https://github.com/sephib/dask_pyconil2019</span></div>


In summary - I hope this talk highlited some of pithalls that can occour when moving from Pandas to Dask.  
remember that Dask is lazy thus requireing some assistance in order to generate the DAG - i.e. add the *meta* kwarg.  
I hope you noticed the similarity between the Pandas code and the Dask code.  

once transfered you enjoy the ease for potential out scaling  in addition to the bonus dashboard for monitoring and profiling of the tasks

beware of the documenation and missing functionalities and the complexity in the DAG

